In [1]:
import math
import requests
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

def numStr(num):
    if num >= 10:
        return str(num)
    else:
        return '0' + str(num)

Base = "http://race.sp.netkeiba.com/?pid=race_result&race_id="
dst = ''

df = pd.DataFrame()


## レース結果のurlはrace_id={西暦}{競技場No}{第何回開催か}{何日目か}{何ラウンドか}で表される。
## 強引にfor文でstringを結合してぶん回す
## 202106030311

for year in tqdm(range(2017, 2019)):
    for i in tqdm(range(1,11)):  #競馬場No
        for j in range(1,11):  #n回
            for k in range(1,9):  #n日目
                for l in range(1,13):  #1~12R
                    url = Base + str(year) + str(i).zfill(2) + '%02d'%j + '{:02}'.format(k) + f'{l:02}'  ##あえて全部違う方式で書いてみた
                    time.sleep(0.5)  ##  0.5秒待機(HPへの労り)
                    html = requests.get(url)  ##　urlのページのhtmlをゲット
                    html.encoding = 'EUC-JP'  ##  unicordからEUC-JPに

                    # scraping
                    soup = BeautifulSoup(html.text, 'html.parser')  ##パースする
                    # ページがあるかの判定
                    if soup.find_all('div', attrs={'class', 'Result_Guide'})!=[]:
                        break
                    else:
                        #共通部分を抜き出す(馬に関係ない今回のレースの共通部分)
                        
                        CommonYear = year  ##開催年
                        CommonDate = soup.find('div', attrs={'class', 'Change_Btn'}).get_text(strip=True)  ##開催日
                        CommonField= soup.find('div', attrs={'class', 'Change_Btn Course'}).get_text(strip=True)  ##開催場所
                        CommonRace = soup.find('div', attrs={'class', 'Race_Num'}).get_text(strip=True)  ##ラウンド
                        CommonRname= soup.find('dt', attrs={'class', 'Race_Name'}).get_text(strip=True)  ##レース名
                        Racedata = soup.find('dd', attrs={'class', 'Race_Data'})
                        CommonCourse= Racedata.find('span').get_text(strip=True)  ##芝かダートか
                        CommonLong= Racedata.find_all('span')[1].get_text(strip=True)  ##
                        CommonRorL=str(Racedata.get_text(strip=True).split()[0])[-2]  ##右回りか左回りか
                        CommonCount=len(soup.find_all('div', attrs='Rank'))  ##頭数   
                        CommonWeather=Racedata.find('span', attrs={'class', 'WeatherData'}).get_text(strip=True)  ##天気
                        CommonGround=Racedata.find('span', attrs={'class', re.compile('Item.*')}).get_text(strip=True)  ##馬場
                        CommonMoney=soup.find('div', attrs={'class', 'RaceDetail fc'}).find('p').get_text(strip=True)
    

                        for m in range(len(soup.find_all('div', attrs='Rank'))):  ##1着から最終着まで回す
                            dst = pd.Series()
                            try:
                                dst['year'] = CommonYear
                                dst['date'] = CommonDate
                                dst['field']= CommonField
                                dst['race'] = CommonRace
                                dst['race_name'] = CommonRname
                                dst['course'] = CommonCourse
                                dst['head_count'] = CommonCount
                                dst['Long'] = CommonLong
                                dst['RorL'] = CommonRorL
                                dst['Weather'] = CommonWeather
                                dst['Ground'] = CommonGround

                                dst['rank'] = soup.find_all('div', attrs={'class', 'Rank'})[m].contents[0] ##着順
                                dst['waku'] = soup.find_all('td',attrs={'class', re.compile('Num Waku.*')})[2*m].find_all('div')[0].get_text(strip=True)
                                dst['lane'] = soup.find_all('td',attrs={'class', re.compile('Num Waku.*')})[2*m+1].find_all('div')[0].get_text(strip=True)
                                dst['horse_name'] = soup.find_all('dt', attrs={'class', 'Horse_Name'})[m].find('a').get_text(strip=True)

                                detailL = soup.find_all('span', attrs=['class', 'Detail_Left'])[m]
                                dst['gender'] = list(detailL.contents[0].split()[0])[0]
                                dst['age'] = list(detailL.contents[0].split()[0])[1]
                                dst['weight'] = detailL.find('span').get_text(strip=True).split('kg')[0] ##馬体重
                                dst['c_weight'] = detailL.find('span').get_text(strip=True).split('kg')[1].strip('(').strip(')') ##体重変動
                                detailR = soup.find_all('span', attrs=['class', 'Detail_Right'])[m]
                                if len(detailR.contents[0])==1: ## NaNならば
                                    dst['jackie'] = detailR.contents[1].get_text(strip=True)
                                    dst['kinryo'] = detailR.contents[2]
                                    dst['trainerA'] = str(detailR.contents[4].split('･')[0][1:])
                                    dst['trainerB'] = str(detailR.contents[4].split('･')[1].strip('n'))[:-1]
                                else:
                                    dst['jackie'] = detailR.contents[0].split(' ')[1]
                                    dst['kinryo'] = detailR.contents[0].split(' ')[2]
                                    dst['trainerA'] = str(detailR.contents[2].split('･')[0].strip('n'))[1:]
                                    dst['trainerB'] = str(detailR.contents[2].split('･')[1].strip('n'))[:-1]


                                Odds = soup.find_all('td', attrs=['class', 'Odds'])[m]
                                dst['odds'] = Odds.find('dt').get_text(strip=True).strip('倍')
                                dst['ninki'] = Odds.find('dd').get_text(strip=True).strip('人気')

                                Time = soup.find_all('td', attrs={'class', 'Time'})[m+1]
                                dst['TotalTime'] = Time.find('dt').get_text(strip=True)
                                dst['Last3R'] = Time.find_all('dd')[-1].get_text(strip=True).strip('(').strip(')')

                                dst.name = str(year) + numStr(i) + numStr(j) + numStr(k) + numStr(l) + numStr(m)

                                df = df.append(dst)
                            except:
                                pass

df.to_csv('keiba_2017-18.csv', encoding='shift-jis')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:16<01:55, 16.57s/it]


 25%|██▌       | 2/8 [00:32<01:38, 16.43s/it]


 38%|███▊      | 3/8 [00:50<01:23, 16.74s/it]


 50%|█████     | 4/8 [01:08<01:08, 17.09s/it]


 62%|██████▎   | 5/8 [01:26<00:52, 17.38s/it]


 75%|███████▌  | 6/8 [01:43<00:35, 17.50s/it]


 88%|████████▊ | 7/8 [01:44<00:12, 12.46s/it]


100%|██████████| 8/8 [01:45<00:00, 13.16s/it]


 10%|█         | 1/10 [01:45<15:47, 105.27s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:17<02:04, 17.77s/it]


 25%|██▌       | 2/8 [00:35<01:46, 17.70s/it]


 38%|███▊      | 3/8 [00:53<01:28, 17.71s/it]


 50%|█████     | 4/8 [01:11<01:11, 17.90s/it]


 62%|██████▎   | 5/8 [01:30<00:54, 18.25s/it]


 75%|███████▌  | 6/8 [01:49<00:36, 18.48s/it]


 88%|████████▊ | 7/8 [01:50<00:13, 13.14s/it]


100%|██████████| 8/8 [01:50<00:00, 13.86s/it]


 20%|██        | 2/10

 25%|██▌       | 2/8 [00:01<00:04,  1.48it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.48it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.48it/s]


 62%|██████▎   | 5/8 [00:03<00:02,  1.48it/s]


 75%|███████▌  | 6/8 [00:04<00:01,  1.49it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.51it/s]


100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


 50%|█████     | 5/10 [03:49<03:21, 40.23s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.51it/s]


 25%|██▌       | 2/8 [00:01<00:03,  1.50it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.49it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.50it/s]


 62%|██████▎   | 5/8 [00:03<00:02,  1.50it/s]


 75%|███████▌  | 6/8 [00:04<00:01,  1.49it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.49it/s]


100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


 60%|██████    | 6/10 [03:54<01:59, 29.77s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.50it/s]


 25%|██▌       | 2/8 [00:01<00:03,  1.50it/s]


 38%|█

 75%|███████▌  | 6/8 [00:04<00:01,  1.49it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.50it/s]


100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


 90%|█████████ | 9/10 [07:19<00:20, 20.23s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.47it/s]


 25%|██▌       | 2/8 [00:01<00:04,  1.48it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.49it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.49it/s]


 62%|██████▎   | 5/8 [00:03<00:02,  1.49it/s]


 75%|███████▌  | 6/8 [00:04<00:01,  1.49it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.49it/s]


100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


100%|██████████| 10/10 [07:24<00:00, 44.50s/it]

 30%|███       | 3/10 [16:00<36:41, 314.49s/it]

  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:21<02:27, 21.12s/it]


 25%|██▌       | 2/8 [00:40<02:04, 20.67s/it]


 38%|███▊      | 3/8 [01:01<01:43, 20.71s/it]


 50%|█████     | 4/8 [01:23<01:23, 20.98s/it]


 62%|██████▎

100%|██████████| 8/8 [03:22<00:00, 25.34s/it]


 30%|███       | 3/10 [09:51<22:51, 195.94s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:26<03:03, 26.16s/it]


 25%|██▌       | 2/8 [00:51<02:34, 25.81s/it]


 38%|███▊      | 3/8 [01:16<02:08, 25.76s/it]


 50%|█████     | 4/8 [01:41<01:41, 25.30s/it]


 62%|██████▎   | 5/8 [02:06<01:16, 25.37s/it]


 75%|███████▌  | 6/8 [02:31<00:50, 25.19s/it]


 88%|████████▊ | 7/8 [02:54<00:24, 24.68s/it]


100%|██████████| 8/8 [03:20<00:00, 25.06s/it]


 40%|████      | 4/10 [13:12<19:43, 197.29s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:25<02:55, 25.10s/it]


 25%|██▌       | 2/8 [00:50<02:31, 25.33s/it]


 38%|███▊      | 3/8 [01:16<02:07, 25.49s/it]


 50%|█████     | 4/8 [01:42<01:41, 25.42s/it]


 62%|██████▎   | 5/8 [02:08<01:17, 25.80s/it]


 75%|███████▌  | 6/8 [02:36<00:53, 26.51s/it]


 88%|████████▊ | 7/8 [03:04<00:26, 26.80s/it]


100%|██████████| 8/8 [03:31<00:00, 26.38s/it]


 50%

 25%|██▌       | 2/8 [00:01<00:03,  1.51it/s]


 38%|███▊      | 3/8 [00:01<00:03,  1.51it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.51it/s]


 62%|██████▎   | 5/8 [00:03<00:01,  1.51it/s]


 75%|███████▌  | 6/8 [00:03<00:01,  1.51it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.51it/s]


100%|██████████| 8/8 [00:05<00:00,  1.50it/s]


 80%|████████  | 8/10 [18:52<02:41, 80.54s/it] 


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.50it/s]


 25%|██▌       | 2/8 [00:01<00:04,  1.50it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.49it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.49it/s]


 62%|██████▎   | 5/8 [00:03<00:02,  1.47it/s]


 75%|███████▌  | 6/8 [00:04<00:01,  1.48it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.48it/s]


100%|██████████| 8/8 [00:05<00:00,  1.49it/s]


 90%|█████████ | 9/10 [18:58<00:57, 58.00s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.49it/s]


 25%|██▌       | 2/8 [00:01<00:04,  1.49it/s]


 38%|

...

 75%|███████▌  | 6/8 [00:04<00:01,  1.49it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.49it/s]


100%|██████████| 8/8 [00:05<00:00,  1.48it/s]


 90%|█████████ | 9/10 [13:41<00:34, 34.27s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.51it/s]


 25%|██▌       | 2/8 [00:01<00:03,  1.50it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.50it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.50it/s]


 62%|██████▎   | 5/8 [00:03<00:02,  1.50it/s]


 75%|███████▌  | 6/8 [00:03<00:01,  1.50it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.50it/s]


100%|██████████| 8/8 [00:05<00:00,  1.50it/s]


100%|██████████| 10/10 [13:46<00:00, 82.69s/it]

 30%|███       | 3/10 [28:58<1:06:28, 569.73s/it]

  0%|          | 0/10 [00:00<?, ?it/s]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:35<04:11, 35.88s/it]


 25%|██▌       | 2/8 [01:12<03:37, 36.20s/it]


 38%|███▊      | 3/8 [01:49<03:01, 36.27s/it]


 50%|█████     | 4/8 [02:42<02:45, 41.26s/it]


 62%|█████

100%|██████████| 8/8 [06:04<00:00, 45.61s/it]


 30%|███       | 3/10 [17:59<41:21, 354.45s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:43<05:04, 43.48s/it]


 25%|██▌       | 2/8 [01:26<04:20, 43.34s/it]


 38%|███▊      | 3/8 [02:12<03:40, 44.13s/it]


 50%|█████     | 4/8 [02:56<02:56, 44.15s/it]


 62%|██████▎   | 5/8 [03:39<02:10, 43.64s/it]


 75%|███████▌  | 6/8 [04:23<01:27, 43.86s/it]


 88%|████████▊ | 7/8 [05:06<00:43, 43.49s/it]


100%|██████████| 8/8 [05:51<00:00, 43.97s/it]


 40%|████      | 4/10 [23:51<35:21, 353.65s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:40<04:42, 40.30s/it]


 25%|██▌       | 2/8 [01:19<03:59, 39.94s/it]


 38%|███▊      | 3/8 [02:07<03:31, 42.27s/it]


 50%|█████     | 4/8 [02:49<02:49, 42.44s/it]


 62%|██████▎   | 5/8 [03:38<02:12, 44.13s/it]


 75%|███████▌  | 6/8 [04:24<01:29, 44.94s/it]


 88%|████████▊ | 7/8 [05:14<00:46, 46.31s/it]


100%|██████████| 8/8 [05:55<00:00, 44.40s/it]


 50%

 25%|██▌       | 2/8 [00:01<00:04,  1.49it/s]


 38%|███▊      | 3/8 [00:02<00:03,  1.49it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.50it/s]


 62%|██████▎   | 5/8 [00:03<00:01,  1.51it/s]


 75%|███████▌  | 6/8 [00:03<00:01,  1.50it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.50it/s]


100%|██████████| 8/8 [00:05<00:00,  1.50it/s]


 80%|████████  | 8/10 [32:59<04:35, 137.99s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.52it/s]


 25%|██▌       | 2/8 [00:01<00:03,  1.52it/s]


 38%|███▊      | 3/8 [00:01<00:03,  1.52it/s]


 50%|█████     | 4/8 [00:02<00:02,  1.52it/s]


 62%|██████▎   | 5/8 [00:03<00:01,  1.53it/s]


 75%|███████▌  | 6/8 [00:03<00:01,  1.53it/s]


 88%|████████▊ | 7/8 [00:04<00:00,  1.52it/s]


100%|██████████| 8/8 [00:05<00:00,  1.52it/s]


 90%|█████████ | 9/10 [33:04<01:38, 98.17s/it] 


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:00<00:04,  1.52it/s]


 25%|██▌       | 2/8 [00:01<00:03,  1.51it/s]


 38%

 50%|█████     | 4/8 [03:23<03:20, 50.03s/it]


 62%|██████▎   | 5/8 [04:16<02:32, 50.82s/it]


 75%|███████▌  | 6/8 [05:08<01:42, 51.26s/it]


 88%|████████▊ | 7/8 [06:02<00:51, 52.00s/it]


100%|██████████| 8/8 [06:52<00:00, 51.53s/it]


 20%|██        | 2/10 [12:57<50:35, 379.45s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:54<06:19, 54.21s/it]


 25%|██▌       | 2/8 [01:49<05:26, 54.45s/it]


 38%|███▊      | 3/8 [02:45<04:34, 54.89s/it]


 50%|█████     | 4/8 [03:40<03:39, 54.91s/it]


 62%|██████▎   | 5/8 [04:38<02:47, 55.95s/it]


 75%|███████▌  | 6/8 [05:25<01:46, 53.32s/it]


 88%|████████▊ | 7/8 [06:19<00:53, 53.37s/it]


100%|██████████| 8/8 [07:11<00:00, 53.94s/it]


 30%|███       | 3/10 [20:09<46:05, 395.06s/it]


  0%|          | 0/8 [00:00<?, ?it/s]


 12%|█▎        | 1/8 [00:47<05:30, 47.16s/it]


 25%|██▌       | 2/8 [01:30<04:35, 45.97s/it]


 38%|███▊      | 3/8 [02:17<03:51, 46.38s/it]


 50%|█████     | 4/8 [03:06<03:08, 47.15s/it]


 62%

ConnectionError: HTTPConnectionPool(host='race.sp.netkeiba.com', port=80): Max retries exceeded with url: /?pid=race_result&race_id=201808050810 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa06862dcd0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
print(df)